# Helpers

In [10]:
from estimators.bandits import clopper_pearson, gaussian, cressieread
import scipy
import random


def _evaluate(data, estimator, alpha):
    for _, ex in data.iterrows():
        estimator.add_example(**ex)
    return estimator.get(alpha)


def evaluate(data, rmin, rmax, alpha):
    return [
        (
            "cressieread",
            _evaluate(data, cressieread.Interval(rmin=rmin, rmax=rmax), alpha),
        ),
        ("gaussian", _evaluate(data, gaussian.Interval(), alpha)),
        ("clopper_pearson", _evaluate(data, clopper_pearson.Interval(), alpha)),
    ]

# Define simulator

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import *


def simulator(n, epsilon, variance):
    for i in range(n):
        chosen = i % 2
        noise = (random.random() - 0.5) * variance
        yield {
            "p_log": 0.5,
            "p_pred": 1 - epsilon if chosen == 0 else epsilon,
            "r": 1 + noise if chosen == 0 else noise,
        }


class CiPlayground:
    def __init__(self):
        def test(**kwargs):
            rmin = kwargs.pop("rmin")
            rmax = kwargs.pop("rmax")
            alpha = kwargs.pop("alpha")
            self.data = pd.DataFrame(simulator(**kwargs))
            self.result = evaluate(self.data, rmin=rmin, rmax=rmax, alpha=alpha)
            plt.boxplot(
                [r[1] for r in self.result], whis=0, labels=[r[0] for r in self.result]
            )

        interact(
            test,
            rmin=FloatSlider(min=-100, value=1, max=0, step=0.01),
            rmax=FloatSlider(min=0, value=1, max=10, step=0.01),
            alpha=FloatSlider(min=0, max=1, step=0.01, value=0.05),
            n=IntSlider(min=0, max=100, value=1000),
            epsilon=FloatSlider(min=0, max=1, step=0.1, value=0.1),
            variance=FloatSlider(min=0, max=10, value=0),
        )

# Interact

In [45]:
playground = CiPlayground()

interactive(children=(FloatSlider(value=0.0, description='rmin', max=0.0, min=-100.0, step=0.01), FloatSlider(…

In [46]:
playground.result

[('cressieread', [0.8811755457962163, 1.2347767819839865]),
 ('gaussian', [0.867188624508888, 1.1818234382689699]),
 ('clopper_pearson', [nan, nan])]